# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [12]:
import requests

url = "https://www.imdb.com/search/title"
response = requests.get(url)
response

<Response [403]>

In [6]:

# Función para convertir el ingreso bruto a millones de dólares
def convert_gross_to_millions(gross):
    if gross:
        gross = gross.replace('$', '').replace('M', '').replace(',', '').strip()
        try:
            return float(gross)
        except:
            return None
    return None

# Función principal para realizar el scraping
def scrape_imdb(title_type, user_rating, start_date, end_date):
    base_url = "https://www.imdb.com/search/title/"
    
    # Parámetros de la URL
    params = {
        'title_type': title_type,
        'release_date': f'{start_date},{end_date}',
        'user_rating': f'{user_rating},',
        'countries': 'us',  # Filtra solo por películas de EE. UU.
        'languages': 'en',
        'count': 250  # Máximo de títulos a mostrar por página
    }
    
    # Realiza la solicitud a IMDb
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Contenedores de películas
    movies_list = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Listas para almacenar los datos
    movie_data = []
    
    # Iterar a través de cada película
    for idx, movie in enumerate(movies_list, start=1):
        # Extraer título
        header = movie.find('h3', class_='lister-item-header')
        title = header.find('a').text
        year = header.find('span', class_='lister-item-year').text
        year = re.sub(r'[^\d]', '', year)  # Extraer solo el año
        
        # Extraer calificación
        rating_tag = movie.find('div', class_='inline-block ratings-imdb-rating')
        rating = float(rating_tag['data-value']) if rating_tag else None
        
        # Extraer duración
        runtime_tag = movie.find('span', class_='runtime')
        runtime = int(runtime_tag.text.replace('min', '').strip()) if runtime_tag else None
        
        # Extraer género
        genre_tag = movie.find('span', class_='genre')
        genre = genre_tag.text.strip() if genre_tag else None
        
        # Extraer descripción
        description_tag = movie.find_all('p', class_='text-muted')
        description = description_tag[1].text.strip() if len(description_tag) > 1 else None
        
        # Extraer director y estrellas
        people = movie.find('p', class_='').text.strip().split('|')
        director = people[0].replace('Director:', '').replace('Directors:', '').strip() if people else None
        stars = people[1].replace('Stars:', '').strip() if len(people) > 1 else None
        
        # Extraer votos
        votes_tag = movie.find('span', {'name': 'nv'})
        votes = int(votes_tag['data-value']) if votes_tag else None
        
        # Extraer ingresos brutos
        gross_tag = movie.find_all('span', {'name': 'nv'})
        gross = convert_gross_to_millions(gross_tag[1]['data-value']) if len(gross_tag) > 1 else None
        
        # Guardar la información en una lista
        movie_data.append({
            'Movie Nr': idx,
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Runtime (min)': runtime,
            'Genre': genre,
            'Description': description,
            'Director': director,
            'Stars': stars,
            'Votes': votes,
            'Gross ($M)': gross
        })
    
    # Convertir los datos a un DataFrame de pandas
    movie_df = pd.DataFrame(movie_data)
    
    return movie_df

# Uso del script con los parámetros solicitados
df = scrape_imdb(title_type='feature', user_rating='7.5', start_date='1990', end_date='1992')

# Mostrar las primeras filas del DataFrame resultante
print(df.head())


Empty DataFrame
Columns: []
Index: []


In [10]:
pip install requests BeautifulSoup

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\nana-\AppData\Local\Temp\pip-install-qcj3ut6q\beautifulsoup_73eed5f6fde64bec96feee405db4b639\setup.py", line 3
          "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                         ^^
      SyntaxError: invalid syntax
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an is

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here